# Base Model: Logistic Regression

## Date: Nov 9, 2023

------------------

## Introduction

In this notebook, a baseline classification model will be implemented using Logistic Regression. Log reg is a good baseline as it is one of the simplest classification models, offers high explainability, and is computationally light compared some of to its counterparts. 

The steps:  
After the data is read in, basic assumptions are established and checked. Any features that show high colinearity and multicollinearity are removed. Then 3 iterations of log reg will be run.
1. Unbalanced unscaled dataset: Demonstrate how balancing the dataset affects the model's performance. 
2. Balanced scaled dataset. The first baseline model. No hyperparameter tuning
3. Optimized model. A optimized model will be used by varying the solver, iterations, regulatization to achieve the best stable baseline log reg model.

Evaluation will be focused on the 2nd and 3rd iterations as those showed significant improvements.  

Note:
- n_jobs can be increased if you have more cpu cores to speed up the modelling
- vif calculations and the gridsearch at the end can take a few minutes to run

----------------

### Table of Contents

1. [Introduction](#Introduction)
   - [Table of Contents](#Table-of-contents)
   - [Import Librarys](#Import-Librarys)
   - [Data Dictionary](#Data-Dictionary)
   - [Define Functions](#Define-Functions)
   - [Load the data](#Load-the-data)
3. [Logistic Regression Model](#Logistic-Regression-Model)
   - [Assumptions](#Assumptions)
   - [PreProcessing](#PreProcessing)
   - [Modelling](#Modelling)
   - [Evaluation](#Evaluation)
8. [Conclusion](#Conclusion)


### Import Librarys

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path
from joblib import dump

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_selector
from sklearn.metrics import roc_curve, auc, roc_auc_score

from statsmodels.stats.outliers_influence import variance_inflation_factor

from helpers import display_corr_heatmap, data_dict

### Data Dictionary

Display the data dictionary. Reminder to place it under the `/Data/Lending_club/` directory like the data.

In [ ]:
data_dict()

### Load the Data

In [ ]:
# Define the relative path to the file
parquet_file_path = Path('../Data/Lending_club/model_cleaned')

try:
    # Read the parquet file
    loans_df = pd.read_parquet(parquet_file_path)
except FileNotFoundError as e:
    print(e.args[1])
    print('Check file location')

In [ ]:
loans_df.head()

## Logistic Regression Model

-------------

### Assumptions 

Before we can start modeling, some base assumptions must be met in order to use a log reg model.   
These include:  
* **Binary Outcome:** The dependent variable is a binary. This is met as loan status has been encoded as 1 and 0
* **Independence:** It is reasonable to assume loans are independent. Without identifiable information, there is not way of knowing from the dataset whether a borrower has applied for multiple loans as the member_id data has been removed by lendingclub.
* **No collinearity / multicollinearity.** Can lead to unstable coefficient estimates and interpretation difficulty. This will be checked
* **Sufficiently Large sample size:** More than 100,000 loans. This is met

### Colinearity

Plot a correlation heatmap for the remaining features.

In [ ]:
display_corr_heatmap(loans_df)

There are some features with high colinearity, such as `open_acc`, `total_acc`, etc. These are primarily from the applicants credit report.

### Collinearity / Multicollinearity

Check for multicollinearity and collineartiy before we split the data or encode categorical variables. First check for multicollinearity using Variance Inflation Factor (VIF). 

Create a df to hold the vif scores. Calculate the vif scores for each feature and place in the dataframe. This may take a few moments as it is running a linear regression for each feature against all others.

In [ ]:
%%time
#define a vif threshold. 5 and 10 are standard 
vif_cutoff = 10

#select only numeric features
numeric_df = loans_df.select_dtypes(include=[np.number])

#create a dataframe to hold the vif scores for each feature
vif_data = pd.DataFrame()
vif_data['feature'] = numeric_df.columns

#calculate the vif
print('Running vif calculations...')
vif_data['VIF'] = [variance_inflation_factor(numeric_df.values, i) for i in range(len(numeric_df.columns))]
print('Finished vif calculations')

In [ ]:
vif_data.sort_values(by=['VIF'], ascending=False)

Create a list of the columns with a vif greater than the threshold

In [ ]:
high_vif_columns = vif_data[vif_data['VIF'] > vif_cutoff]['feature'].tolist()

Before dropping the features with high vif, inspect them

In [ ]:
display(high_vif_columns)

We will leave `loan_amnt`, `term`, `int_rate`, as these are key features of the dataset.

In [ ]:
# Drop features with high VIF
filtered_high_vif_columns = [feature for feature in high_vif_columns if feature not in ['loan_amnt', 'term', 'int_rate']]
loans_df.drop(columns = filtered_high_vif_columns, inplace=True)

The remaining features:

In [ ]:
loans_df.head(0)

***Collinearity***

In [ ]:
display_corr_heatmap(loans_df)

In [ ]:
loans_df.drop(columns = ['num_tl_op_past_12m'], inplace=True)

There are still some high correlations between variables specifically `num_tl_op_past_12m`, the number of accounts opened in the past 12 months, with
`acc_open_past_24mths`, the number of trades opened the last 24 months. Although the correlation is moderately high, there is no redundancy as they represent different aspects of the credit report, so both will be kept. There are no major features highly correlated with the target variable.  
All the assumptions have now been met

### PreProcessing

***Train Test Split***

In [ ]:
# Split the data
X = loans_df.drop(columns=['loan_status'], inplace=False)
y = loans_df['loan_status']

# Split into train and test sets. Stratify to ensure any inbalance is preserved as in the original data and not lost by random chance. 
# Create seperate splits for the unbalanced and balanced model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=11, stratify=y)
unbal_X_train, unbal_X_test, unbal_y_train, unbal_y_test = train_test_split(X, y, test_size=0.3, random_state=11, stratify=y)

***Data Inbalance***

As shown in EDA, there is a large inbalance between the number of successful loans (class 1) and failed loans (class 0), approximately 80/20. Since the dataset is sufficiently large, it is acceptable to downsample the instances of class 1 to equal class 0. Balancing the dataset reduces the risk of any bias introduced to a single class simply due to its frequency in the dataset.  

This also has the added advantage of giving a more manageable dataset size. However, if computation power is not an issue, then more failed loans could be sampled from the original dataset, and / or synthetic data created for the minority class using SMOTE.  
More information can be found here:  
https://towardsdatascience.com/smote-fdce2f605729

In [ ]:
print('Number of class 1 examples before:', X_train[y_train == 1].shape[0])

# Downsample majority class without replacement to the same size of the minority class
X_downsampled, y_downsampled  = resample(X_train[y_train == 1],
                                   y_train[y_train == 1],
                                   replace=False,
                                   n_samples=X_train[y_train == 0].shape[0],
                                   random_state=1)

print('\nNumber of class 1 examples after:', X_downsampled.shape[0])

Can now combine with the original dataset.

In [ ]:
# Combine the downsampled successful loans with the failed loans. Will keep as a df since changing to 
X_train_bal = pd.concat([X_train[y_train == 0], X_downsampled])
y_train_bal = np.hstack((y_train[y_train == 0], y_downsampled))

print("New X_train shape: ", X_train_bal.shape)
print("New y_train shape: ", y_train_bal.shape)
print("X_test shape: ", X_test.shape)
print("y_test shape: ", y_test.shape)

***Inspect Categorical Features***

Inspect whether the categorical features are ordinal or nominal. 

In [ ]:
categorical_columns = X_train_bal.select_dtypes('object').columns.tolist()
display(categorical_columns)
categorical_columns.remove('verification_status')

In [ ]:
X_train_bal['verification_status'].value_counts()

The feature `verification_status` will be **ordinal encoded** since loan applications with verified income information should be weighted higher than those that are only source verified or unverified. The other categorical features can be onehot encoded.

***Column Transformation for 1st iteration***

Just encode the categorical variables as described above.

In [ ]:
#instantiate onehot encoder
unbal_categorical_transformer = OneHotEncoder(handle_unknown='ignore')

#instantiate ordinal encoder
unbal_ordinal_transformer = OrdinalEncoder(categories=[['Not Verified', 'Source Verified', 'Verified']])

#instantiate the column transformer
unbal_preprocessor = ColumnTransformer(
    transformers=[
        ('cat', unbal_categorical_transformer, ['home_ownership', 'purpose', 'application_type']),
        ('ord', unbal_ordinal_transformer, ['verification_status']),
    ],
    remainder='passthrough',
    n_jobs=2 #use 2 cpu cores for greater speed
)

#fit to the train set
unbal_preprocessor.fit(unbal_X_train)

#transform the train and test sets
unbal_X_train_transformed = unbal_preprocessor.transform(unbal_X_train)
unbal_X_test_transformed = unbal_preprocessor.transform(unbal_X_test)

print("Shape of train transformed: ", unbal_X_train_transformed.shape)
print("Shape of test transformed: ",  unbal_X_test_transformed.shape)

***Column Transformation for 2nd Iteration***

For the second iteration, a standard scaler is fit as well. Although log reg is not a distance based model, it can aid in model performance by reducing the size of the parameter space, allowing the model to converge more easily.  
More information can be found here:  
https://forecastegy.com/posts/does-logistic-regression-require-feature-scaling/

In [ ]:
#instantiate onehot encoder
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

#instantiate ordinal encoder
ordinal_transformer = OrdinalEncoder(categories=[['Not Verified', 'Source Verified', 'Verified']])

#combine into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, ['home_ownership', 'purpose', 'application_type']),
        ('ord', ordinal_transformer, ['verification_status']),
        ('num', StandardScaler(), make_column_selector(dtype_include=['int64','int32','float64','float32']))
    ],
    remainder='passthrough',
    n_jobs=2
)

#fit to the train set
preprocessor.fit(X_train_bal)

#transform the train and test sets
X_train_transformed = preprocessor.transform(X_train_bal)
X_test_transformed = preprocessor.transform(X_test)

print("Shape of train transformed: ", X_train_transformed.shape)
print("Shape of test transformed: ", X_test_transformed.shape)

***Run the model 1st iteration***

The log reg models are ready to be run. A log reg model will be run on both the balanced downsampled data as well as the inbalanced data, to showcase model evaluation wrt to class balance. The log reg model will use the `lbfgs` solver as it performs well on small dataset, even though it may not converge. If the model does not converge, we will check for any features with high multicollinearity, and try a different solver and/ or higher iteration count.  

In [ ]:
# Initializing and training the logistic regression model
unbal_log_reg = LogisticRegression(random_state=1,
                             solver='lbfgs', 
                             max_iter=4000, 
                             verbose=2, #output while the model runs
                             n_jobs=2) #use 2 cpu cores
                             #class_weight='balanced') #weight the minority class to counter inbalance also works

unbal_log_reg.fit(unbal_X_train_transformed, unbal_y_train)

# Making predictions on the test data using the trained model
unbal_y_pred = unbal_log_reg.predict(unbal_X_test_transformed)

***Score the 1st iteration model***

In [ ]:
# Score the model on both train and test data
unbal_train_score = unbal_log_reg.score(unbal_X_train_transformed, unbal_y_train)
unbal_test_score = unbal_log_reg.score(unbal_X_test_transformed, unbal_y_test)

conf_matrix = confusion_matrix(unbal_y_test, unbal_y_pred)
class_report = classification_report(unbal_y_test, unbal_y_pred)

# Display the Confusion Matrix
ConfusionMatrixDisplay.from_estimator(
    unbal_log_reg, 
    unbal_X_test_transformed, 
    unbal_y_test, 
    cmap='Blues', 
    display_labels=['Class 0', 'Class 1']
)
plt.title('Confusion Matrix for Logistic Regression')
plt.show()

#################################
# Calculate the ROC curve

# Calculate the probability scores
probs = unbal_log_reg.predict_proba(unbal_X_test_transformed)

# Keep only the positive class (class 1)
probs = probs[:, 1]

# Pull out the fprs, tprs, and thresholds
fprs, tprs, thresholds = roc_curve(unbal_y_test, probs)
roc_auc = roc_auc_score(unbal_y_test, probs)

# Plot the ROC curve
plt.figure()
plt.plot(fprs, tprs, color='darkorange',
         lw=2, label='AUC = %0.2f' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('ROC Curve for Loan Default Prediction')
plt.legend(loc="best")
plt.show()

print(f'Area under curve (AUC):{roc_auc}')
#################################

# Print the confusion matrix and Classification Report
print("-"*20)
print("Confusion matrix:")
print(conf_matrix)
print("-"*20)
print(class_report)
print("-"*20)

print(f'Score on train: {unbal_train_score}')
print(f'Score on test: {unbal_test_score}')
print("-"*20)

As a scaled balanced dataset will almost guaranteed perform better, a brief evaluation on the confusion matrix and classification report will be sufficient to justify balancing the dataset. At first glance, the model seems to have performed well with a ~ 78.8% accuracy for both train and test sets. However, when considered with the class inbalance of 73.7%, a 78.7% accuracy is a small improvement over random guesswork. This inbalance results in the model predicting class 1 more frequently, resulting in a very high recall score for successful loans. Of all the successful loans, 99% were correctly identified, meaning there were very successful lending opportunities were missed. However, this came at a cost. For failed loans, the precision is very low, with a terrible recall. Indiscriminately predicting many loans to be successful led to a high number of false positives, with failed loans being misclassified as successful. This would result in a much higher credit risk. Of all the failed loans, only 3% were correctly identified (recall), with 42% of loans predicted as failed, actually failed (precision). The low precision would result in many missed lending oppertunites, with the low recall resulting in a very high credit risk. 

The weighted combined f1 score, an average for the precision and recall, is moderate at 0.71, showing overall a weak differentiation between successful and failed loans. However, this metric has less value considering the class inbalance. 

The ROC curve shows that the model is doing better than random choice (the horizontal line). The AUC of of 0.64 implies that when randomly selecting one positive and one negative example, the model will correctly assign a higher probability of being positive to the positive example about 64% of the time. Ideally, an AUC score closer to 1 would be preferred since that requires a higher TPR and lower FPR. 

Overall, this model correctly predicted many successful loans, at the cost of classifying many bad loans as successful. 
In the next iteration with a balanced scaled dataset, the aim will be to maintain a comparable precision, while minimizing false positives, resulting in a better balance between real credit opportunities and managing default risks. 

***Run the model 2nd iteration***

In [ ]:
# Initializing and training the logistic regression model
log_reg = LogisticRegression(random_state=1,
                             solver='lbfgs', 
                             max_iter=4000, 
                             verbose=2, #output while the model runs
                             n_jobs=2) #use 2 cpu cores

log_reg.fit(X_train_transformed, y_train_bal)

# Making predictions on the test data using the trained model
y_pred_bal = log_reg.predict(X_test_transformed)

***Score the 2nd iteration model***

In [ ]:
# Score the model on both train and test data
train_score = log_reg.score(X_train_transformed, y_train_bal)
test_score = log_reg.score(X_test_transformed, y_test)

# Evaluating the model with confusion matrix and a classification report
conf_matrix = confusion_matrix(y_test, y_pred_bal)
class_report = classification_report(y_test, y_pred_bal)

#Display the confusion matrix
ConfusionMatrixDisplay.from_estimator(
    log_reg, 
    X_test_transformed, 
    y_test, 
    cmap='Blues', 
    display_labels=['Class 0', 'Class 1']
)
plt.title('Confusion Matrix for Logistic Regression')
plt.show()

print("-"*20)
print("Confusion matrix:")
print(conf_matrix)
print("-"*20)
print(class_report)
print("-"*20)

print(f'Score on train: {train_score}')
print(f'Score on test: {test_score}')

### Interpretation

The model has~ 65.0%% accuracy on the train ste and~ 65.6% accuracy on the test stt. The score between the train and test set are clos,  meaning the model generalizes  well to unseen data, and that there is no overfitting or underfitting ,However, this will be further explored with the classification report 

The confusion matrix above shows the counts for correctly and incorrectly predicted classes, in the format of:  
```
Predicted Label 
    0      1 
+------+------+  
| TP   |  FP  |  0
+------+------+     True Label
| FN   |  TN  |  1
+------+------+  
```


Where,
- **True Negative (TN):**177738 loans were correctly predicted as failed (class 0).
- **False Positive (FP):**100861 cases were incorrectly predicted as successful (class 1) when they are actually failed (class 0).
- **False Negative (FN):**35876 3 cases were incorrectly predicted as failed (class 0) when they are actually successful (class 1).
- **True Positive (TP):**701069 cases were correctly predicted as successful (class 1).  

The model showed a strong ability to discern successful from failed loans, with the majority of successful loans being accurately identified.
For this project, the primary goal is to minimize false positives, ie instance of failed loans incorrectly predicted as successful, minimizing credit default risk. Of the 27,859 failed loans, 17,788 were correctly predicted as failed, and of the 105,982 successful loans, 70,099 were correctly predicted as successful. While the model will be tunedto favor  precision, however, this can be adjusted based on the lenders risk appetite, allowing for a more balanced approach between granting credit and managing default risks.


Classification Report
- **Precision for Class 0:** 0.33, meaning when the model predicts failed, it is correct ~ 33% of the time.
- **Recall for Class 0:** 0.64, meaning that the model correctly identifies ~ 64% of the actual failed cases.
- **F1-Score for Class 0:** 0.44, a weighted average of precision and recall for failed loans, indicating a moderate balance between precision and recall for this class.
- **Support for Class 0:** There are 27,859 actual occurrences of failed loans in the dataset. 

- **Precision for Class 1:** 0.87, suggesting that when the model predicts successful, it is correct ~ 87% of the time.
- **Recall for Class 1:** 0.66, meaning that the model correctly identifies ~ 66% of the actual successful cases.
- **F1-Score for Class 1:** 0.75, n  average of precision and recall for successful loans, indicating a strong balance between precision and recall for the successful class.
- **Support for Class 1:** There are 105,982 actual occurrences of successful loans in the dataset.

Overall Metrics
- **Accuracy:** 0.66, indicating that the overall, the model correctly predicts 66% of the cases.
- **Macro Average Precision:** 0.60, the average precision across both classes.
- **Macro Average Recall:** 0.65, the average recall across both classes.
- **Macro Average F1-Score:** 0.59, the average F1-score across both classes  .
Compared to the previous iteration, this model has made significant improvement. The precision for class 1 increased from 0.79 -> 0.86, meaning fewer bad loans were misclassified. The increase in precision came at the cost of recall, which decreased from 0.99 to 0.66. Fewer good lending opportunities were captured. However, the recall for failed loans increased from 0.03 to 0.64, meaning the model identified many more bad loans correctly. The f1 score decreased slightly, but the AUC increased. The model improved in distinguishing between both classes.


***Threshold Optimization***

In [ ]:
from sklearn.metrics import precision_score, recall_score
import numpy as np
import matplotlib.pyplot as plt

#get the probabilities for the positive class
y_proba = log_reg.predict_proba(X_test_transformed)[:, 1]

# Vary thresholds by 0.05 from 0.05 to 1
thresholds = np.arange(0.05, 1, 0.05)

precisions = []
recalls = []

for threshold in thresholds:
    # Apply threshold
    y_threshold = np.where(y_proba > threshold, 1, 0)
    
    # Calculate precision and recall
    precision = precision_score(y_test, y_threshold)
    recall = recall_score(y_test, y_threshold)
    
    # Append to list
    precisions.append(precision)
    recalls.append(recall)

# Visualize the result
plt.figure(figsize=(10, 6))
plt.plot(thresholds, precisions, label='Precision', marker='o')
plt.plot(thresholds, recalls, label='Recall', marker='o')
plt.title('Precision and Recall scores as a function of the decision threshold')
plt.xlim(0, 1)
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.legend()
plt.grid(True)
plt.show()

As mentioned above, precision is preferred to recall for this project, as we want to only extend credit to worthy borrowers that are less likely to default, even if that comes at the cost of some missed lending opportunities. A threshold of 0.4 or 0.5 is appropriate here as afterward recall begins to steeply decrease. Once again, this can be varied based on lender risk tolerances.

***ROC Curve***

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_proba)

# Calculate the AUC
roc_auc = auc(fpr, tpr)

# Plotting the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

The increased AUC score means for a certain threshold, the FPR to TPR has been improved, meaning the model is able to better discern betweem classes. 

***Feature importance***

Explore which features were most useful in the prediction be inspecting their weights, keeping in mind to look for any missed leaky features remaining. 

In [ ]:
#get the feature weights out. 
feature_weights = pd.DataFrame({
    'Feature': preprocessor.get_feature_names_out(),
    'Coefficient': log_reg.coef_[0]
})

# Sort the features by the absolute value of their coefficient
feature_weights = feature_weights.sort_values(by='Coefficient', ascending=True)

In [ ]:
# Plotting the feature weights
plt.figure(figsize=(12, 12))
plt.barh(feature_weights['Feature'], feature_weights['Coefficient'], color='lightblue')
plt.xlabel('Coefficient Value')
plt.ylabel('Features')
plt.title('Feature Importance')
plt.show()

The categorical features home ownership and loan purpose are the most positively predictive, with the home ownership, interest rate, and loan term being the most negatively predictive.  
Note:  
- "cat prefix indicates it was a categorical variable, and the "remainder" prefix can be ignored. It is left over from the columntransformer. 

***Log odds***

In [ ]:
log_odds = log_reg.coef_[0]
odds = np.exp(log_odds)

feature_names = preprocessor.get_feature_names_out()
odds_df = pd.DataFrame({'Feature': feature_names, 'LogOdds': log_odds, 'OddsRatio': odds})

#sort df by OddsRatio
odds_df = odds_df.sort_values(by='OddsRatio', ascending=False)
odds_df

We can look at the log odds ie for a unit increase in a feature, how do the odds multiply.
For example,

No Home Ownership: The odds of a successful loan are (2.36-1)X100=136% higher whene ownership compared to the bas (some form of house ownership)eline group, holding all other variabl.atio of t. 

***Final Iteration***

For the final iteration, a automated process will be used to fine tune the hyperparameters. This will ensure no steps are missed, no accidental data leakage is made, and is generally more efficient. As found in the last iteration, log reg trained on a balanced scaled dataset out performed a log reg model trained on a unbalanced unscaled dataset. The hyperparameters to be tuned will be:  
1. C value: The inverse of the strength or regulatization. 
2. Penalty type: L1 (lasso) and L2 (ridge). Although there was no overfitting, regularization might still help with feature selection and model stability
3. Solver: Although there was no issue with convergence above, different solvers allow for different penalties. The SAGA solver will be used as it allows for all the different penalties, is efficient on large datasets, and recommended by sklearn.

The column transformers for encoding the categorical variables are created and passed into a pipeline. The pipeline allows these preprocessing transformations to be bundled together into a single object, carried out sequentially, and helps avoid any data leakage. A hyperparameter grid is then established, allowing GridSearchCV to run an exhaustive check on all the different combinations of hyperparameters by traversing this table. This process of checking all the combinations of hyperparameters is then repeated 5 times for each "fold" in the 5 fold cross-validation. The training data is split in k folds, in this case 5, smaller sets with the model being trained and validated on each k times. This gives a more reliable estimation than a single split due to that fact that when splitting randomly, certain hyperparameters might perform better just based on the happenstance of data included in that set. This can be mitigated if it repeated across the entire dataset, ensuring full coverage, and then averaged across all the folds.  

Note:  
- This is very time consuming
- If you selected different data and find the model is not converging, pass a high max_iter as well  
More information:  
https://datascience.stackexchange.com/questions/108792/why-is-the-k-fold-cross-validation-needed  
https://datascience.stackexchange.com/questions/43972/when-should-i-use-standardscaler-and-when-minmaxscaler
https://stackoverflow.com/questions/73479995/solver-lbfgs-supports-only-l2-or-none-penalties-got-l1-penalty

In [ ]:
%%time
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from tempfile import mkdtemp
from sklearn.feature_selection import SelectFromModel


#instantiate categorical encoders
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
ordinal_transformer = OrdinalEncoder(categories=[['Not Verified', 'Source Verified', 'Verified']])

#cache results for increased speed
cachedir = mkdtemp()

#combine into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, ['home_ownership', 'purpose', 'application_type']),
        ('ord', ordinal_transformer, ['verification_status']),
    ],
    remainder='passthrough',
    n_jobs=2) # 2 cpu cores

#changed max iter since default 100 is quite low
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),
    ('feature_selection', SelectFromModel(LogisticRegression(penalty='l1', solver='saga'))),
    ('classifier', LogisticRegression(max_iter=1000, solver='saga', random_state=1))], memory = cachedir)

#define the hyperparameters grid to search
'''
param_grid = [
    {
        'classifier__penalty': ['l1', 'l2', 'elasticnet'],
        'classifier__C': [0.001, 0.007, 0.01, 0.03, 0.1],
        'classifier__l1_ratio': [0.5]  # Only needed if elasticnet is used
    },
    {
        'classifier__penalty': ['none'],
        'classifier__C': [1]  # 'C' is not used when penalty is 'none'
    }
]
'''
param_grid = [
    {
        'classifier__penalty': ['l1', 'l2'],
        'feature_selection__estimator__penalty': ['l1'],
        'classifier__C': [0.005, 0.006, 0.007, 0.008],
        'feature_selection__threshold': ['mean', 'median']  # Example thresholds
        # No 'l1_ratio' should be specified here
    },
    {
        'classifier__penalty': ['none'],
        'feature_selection__threshold': ['mean', 'median'],  # Example thresholds
        'classifier__C': [1]  # 'C' is not used when penalty is 'none'
    }
]


# Setup the GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=10, n_jobs=2)

#fit to the balanced data. Any transformations are handled by the pipeline
grid_search.fit(X_train_bal, y_train_bal)

In [ ]:
# Get the best model object
best_model = grid_search.best_estimator_

In [ ]:
# Best hyperparameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

In [ ]:
# Mean test score for each CV fold
grid_search.cv_results_['mean_test_score']

In [ ]:
# Get the score
grid_search.score(X_test, y_test)

In [ ]:
pd.DataFrame(grid_search.cv_results_).head()

***Save the best model***

In [ ]:
#relative path to models folder
model_file_path = Path('../Models/best_logistic_regression_model.joblib')
dump(best_model, model_file_path)

***Evaluate the best model***

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay, roc_curve, auc, RocCurveDisplay

grid_search.cv_results_['mean_test_score']

# Make predictions on the test data
y_pred = best_model.predict(X_test)
class_report = classification_report(y_test, y_pred)
print(class_report)


# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# Generate the confusion matrix
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=best_model.classes_)
disp.plot()
plt.title('Confusion Matrix for Logistic Regression')
plt.show()

###########################################
# ROC CURVE AUC CURVE

# Get the fpr, tpr, thresholds from the probability predict
y_proba = best_model.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_proba)

# Calculate the AUC
roc_auc = auc(fpr, tpr)

# Plotting the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

Using the gridsearch, there is no improvement over our second iteration. 

***Feature importance***

***Feature Importance Final Iteration***

In [ ]:
# Convert to 1D array
classifier_coefs = best_model.named_steps['classifier'].coef_.flatten()

#which features were kept
selected_features_mask = best_model.named_steps['feature_selection'].get_support()

# Apply the mask to the transformed feature names to get the selected feature names
selected_feature_names = transformed_feature_names[selected_features_mask]

# Use np.where to find indices of features that were not selected
dropped_feature_indices = np.where(selected_features_mask == False)[0]

# Get the names of the dropped features by indexing into the transformed feature names
dropped_feature_names = transformed_feature_names[dropped_feature_indices]

# Print out the dropped features
print("Dropped Features:", dropped_feature_names)

feature_weights_gridsv = pd.DataFrame({
        'Feature': selected_feature_names,
        'Coefficient': classifier_coefs
    })
# Sort the features by the absolute value of their coefficient
feature_weights_gridsv = feature_weights_gridsv.sort_values(by='Coefficient', ascending=True)

In [ ]:
# Plotting the feature weights
plt.figure(figsize=(12, 12))
plt.barh(feature_weights_gridsv['Feature'], feature_weights_gridsv['Coefficient'], color='lightblue')
plt.xlabel('Coefficient Value')
plt.ylabel('Features')
plt.title('Feature Importance')
plt.show()

The model achieved the same scores, however many features with little or no predictive value were dropped.

***Vary the threshold***

Plot the precision recall for different thresholds again

In [ ]:
from sklearn.metrics import precision_score, recall_score
import numpy as np
import matplotlib.pyplot as plt

# Vary thresholds by 0.05 from 0.05 to 1
thresholds = np.arange(0.05, 1, 0.05)

precisions = []
recalls = []

for threshold in thresholds:
    # Apply threshold
    y_threshold = np.where(y_proba > threshold, 1, 0)
    
    # Calculate precision and recall
    precision = precision_score(y_test, y_threshold)
    recall = recall_score(y_test, y_threshold)
    
    # Append to list
    precisions.append(precision)
    recalls.append(recall)

# Visualize the result
plt.figure(figsize=(10, 6))
plt.plot(thresholds, precisions, label='Precision', marker='o')
plt.plot(thresholds, recalls, label='Recall', marker='o')
plt.title('Precision and Recall scores as a function of the decision threshold')
plt.xlim(0, 1)
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.legend()
plt.grid(True)
plt.show()

The Precision and Recall curves are the same as the previous iteration. A threshold of 0.4, 0.5 is suitable.

***Look for common factors in False Positives***

We can examine the false positives next to pinpoint possible problem features for the model, and to see what feature values were potentially leading to the incorrect predictions.

Note: **This is done on the test set, and therefore we CANNOT run another iteration using this information or retroactively change parameters in the pipeline, as that would be leaking data from the test set and could result in inflated metrics and over fitting. This is simply explanatory.**

https://stackoverflow.com/questions/50094999/how-to-return-an-array-of-false-positives-from-a-confusion-matrix-in-scikit-lear

In [ ]:
import pandas as pd
import numpy as np

# Transform X_test so it matches the feature names
X_test_transformed = best_model.named_steps['preprocessor'].transform(X_test)

# Get feature names from the pipeline's preprocessor. This list is the full feature list
feature_names = best_model.named_steps['preprocessor'].get_feature_names_out()

# Convert to df
X_test_transformed_df = pd.DataFrame(X_test_transformed, columns=feature_names)

#reset indicies
X_test_transformed_df.reset_index(drop=True, inplace=True)
y_test = y_test.reset_index(drop=True)

# Identify the false positives
false_positives_mask = (y_pred == 1) & (y_test == 0)

# Filter X_test for false positives
false_positives_df = X_test_transformed_df[false_positives_mask.values]

# Store the possible problem features
possible_problem_features = []

for column in false_positives_df.columns:
        
    # Get the most common occurring value in the column 
    mode_value = false_positives_df[column].mode()[0]
    mode_freq = (false_positives_df[column] == mode_value).mean()
    
    # If the column is made up primarily of a single value
    if mode_freq > 0.9:

        possible_problem_features.append(column)
        
        print(f"Feature: {column}")
        print(f"Mode value: {mode_value}")
        print(f"Frequency of mode: {mode_freq}")
        print('-'*20)

We can see that for false positives, the onehot encoded category `cat__home_ownership_ANY` as example, is primarily composed of 0 values, and occurs with a frequency of close to 100% among false positives. Meaning, the majority of false positives did not have some form of home ownership. 

This alone does not mean the `home_ownership` feature is a problem for the linear model. It's distribution should be analyzed across the full X_test dataset and then compared to the false positive set in order discern any discrepancy between the two. The other onehot encoded categorical values should also be considered along with the "ANY" flag.  

***PDP graphs***

In [ ]:
# Find the index of the feature name
feature_index = list(feature_names_transformed).index('cat__home_ownership_MORTGAGE')

# Plot the PDP using the index
display = PartialDependenceDisplay.from_estimator(
    best_model, X_test_transformed, [feature_index], grid_resolution=50
)
display.plot()
plt.show()


In [ ]:
# Ensure X_test is transformed using the same preprocessing steps as the training data
X_test_transformed = best_model.named_steps['preprocessor'].transform(X_test)

# Double-check the transformed feature names
feature_names_transformed = best_model.named_steps['preprocessor'].get_feature_names_out()
print(feature_names_transformed)

# Now, verify the presence of the feature again
if 'cat__home_ownership_MORTGAGE' in feature_names_transformed:
    print('It is in')
    # If present, plot the PDP
    display = PartialDependenceDisplay.from_estimator(
        best_model, X_test_transformed, ['home_ownership'], grid_resolution=50
    )
    display.plot()
    plt.show()
else:
    print("'cat__home_ownership_MORTGAGE' is not found in the transformed features.")


In [ ]:
# Make sure possible_problem_features only contains names that are in feature_names
possible_problem_features = [f for f in possible_problem_features if f in feature_names]
print(possible_problem_features)

# Now you can iterate over the possible problem features and plot the PDPs
for feature_name in possible_problem_features:
        # Create the display object
        display = PartialDependenceDisplay.from_estimator(
            best_model, X_test_transformed, [feature_name], grid_resolution=50
        )
        # Plot the partial dependence
        display.plot()
        plt.show()

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

for feature in majority_value_columns:
    feature_name = feature
    # Create the display object
    display = PartialDependenceDisplay.from_estimator(best_model, X_test, [feature_name])
    # Plot the partial dependence
    display.plot()
    plt.show()

## Conclusion

***WRITE THE RESULTS OF BEST OUTCOME TO FILE OR CSV TO BE COMPARED WITH LATER***

The logistic regression model performed quite well considering its explainability and ease of use. We achieved a 66% and 65% accuracy on our baseline model. 